In [25]:
%reload_ext autoreload
%autoreload 2

In [26]:
import chess
from playing.utils import board_to_piece_uci_moves
from playing.agents import GPTEloAgent
from lightning_training import LightningGPT

from data_process import tokenizers
from nanoGPT.model import GPTConfig


In [27]:
material_pair_tokenizer = tokenizers.FullMoveEloMaterialPairTokenizer()

material_tokenizer = tokenizers.FullMoveEloMaterialTokenizer()

base_tokenizer = tokenizers.FullMoveTokenizerWithElo()

material_pair_model_config = GPTConfig(
    block_size=604,
    vocab_size=len(material_pair_tokenizer.vocab),
    n_layer=8,
    n_head=8,
    n_embd=512,
    bias=False,
)

material_model_config = GPTConfig(
    block_size=604,
    vocab_size=material_tokenizer.vocab_size,
    n_layer=8,
    n_head=8,
    n_embd=512,
    bias=False,
)

base_model_config = GPTConfig(
    block_size=302,
    vocab_size=base_tokenizer.vocab_size,
    n_layer=8,
    n_head=8,
    n_embd=512,
    bias=False,
)

test_start_token = 21
test_token_step = 2

test_start_token_base = test_start_token // 2 + 1
test_token_step_base = 1

material_checkpoint = "./models/full_training/elo_material_ignore_material_prediction/epoch=9-step=1250000.ckpt"
# material_single_checkpoint = "./models/small_training/elo_material_ignore_material_prediction/epoch=4-step=78125.ckpt"


material_model = LightningGPT.load_from_checkpoint(
    material_checkpoint,
    config=material_model_config,
    test_start_token=test_start_token,
    test_token_step=test_token_step,
    trainig_ignore_first_n_targets=test_start_token,
    training_target_step=2,
    tokenizer=material_tokenizer,
    masked_elo_test=True
)

material_pair_checkpoint = "./models/full_training/elo_material_pair_ignore_material_prediction/epoch=9-step=1250000.ckpt"

material_pair_model = LightningGPT.load_from_checkpoint(
    material_pair_checkpoint,
    config=material_pair_model_config,
    test_start_token=test_start_token,
    test_token_step=test_token_step,
    trainig_ignore_first_n_targets=test_start_token,
    training_target_step=2,
    tokenizer=material_pair_tokenizer,
    masked_elo_test=True
)

base_checkpoint = "./models/full_training/masked_elo/epoch=9-step=1250000.ckpt"

base_model = LightningGPT.load_from_checkpoint(
    base_checkpoint,
    config=base_model_config,
    test_start_token=test_start_token_base,
    test_token_step=test_token_step_base,
    trainig_ignore_first_n_targets=test_start_token_base,
    training_target_step=test_token_step_base,
    tokenizer=base_tokenizer,
    masked_elo_test=True
)

base_no_mask_checkpoint = "./models/full_training/adaptive_elo/epoch=9-step=1250000.ckpt"

base_no_mask_model = LightningGPT.load_from_checkpoint(
    base_no_mask_checkpoint,
    config=base_model_config,
    test_start_token=test_start_token_base,
    test_token_step=test_token_step_base,
    trainig_ignore_first_n_targets=test_start_token_base,
    training_target_step=test_token_step_base,
    tokenizer=base_tokenizer,
    masked_elo_test=True
)

number of parameters: 27.34M
number of parameters: 28.15M
number of parameters: 27.29M
number of parameters: 27.29M


In [94]:
model = material_model.model
agent = GPTEloAgent(model=model, model_elo=1800, use_material_tokens=True)

board = chess.Board()
agent.play(board)

Move.from_uci('e2e4')